In [1]:
from PIL import Image, ImageChops, ImageEnhance
from keras.models import load_model
from keras.preprocessing import image
import numpy as np

Using TensorFlow backend.


In [2]:
def convert_to_ela_image(path, quality):
    filename = path
    resaved_filename = filename.split('.')[0] + '.resaved.jpg'
    ELA_filename = filename.split('.')[0] + '.ela.png'
    
    im = Image.open(filename).convert('RGB')
    im.save(resaved_filename, 'JPEG', quality=quality)
    resaved_im = Image.open(resaved_filename)
    
    ela_im = ImageChops.difference(im, resaved_im)
    
    extrema = ela_im.getextrema()
    max_diff = max([ex[1] for ex in extrema])
    if max_diff == 0:
        max_diff = 1
    scale = 255.0 / max_diff
    
    ela_im = ImageEnhance.Brightness(ela_im).enhance(scale)
    
    return ela_im

In [3]:
from skimage import transform
from keras.models import load_model
def load(filename):
    image = Image.open(filename)
    np_image = convert_to_ela_image(filename, 90)
    np_image = np.array(np_image).astype('float32')/255
    np_image = transform.resize(np_image, (128, 128, 3))
    np_image = np.expand_dims(np_image, axis=0)
    return np_image

In [13]:
model = load_model('saved-models/model-v1-with-casia.h5')

In [19]:
print('++++ REAL IMAGE ++++')
real = load('custom test data/1.jpg')
print(model.predict_classes(real))

print('++++ FAKE IMAGE ++++')
fake = load('custom test data/license.jpg')
print(model.predict_classes(fake))

++++ REAL IMAGE ++++
[0]
++++ FAKE IMAGE ++++
[1]
